# Prepare

In [7]:
import re
import string
import pandas as pd
import numpy as np
import swifter
from bertopic import BERTopic
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer


## stop-words

In [8]:
spanish_punct = string.punctuation + "¡¿«»"
min_cnt = 15
df_stop = pd.read_csv('c:/Users/rmessina/Eli/data/song_lyrics_es_1950_2020_lemmaNouns_counters.csv')
df_stop = df_stop[df_stop['count'] <= min_cnt]

stop_words = df_stop['noun'].values.tolist()
display(f"Stop words loaded: {len(stop_words)} words")

'Stop words loaded: 172549 words'

In [9]:
def remove_stuff(text):
    return re.sub(r"(?<!\w)-|-(?!\w)|[{}]".format(re.escape(spanish_punct.replace("-", ""))), "", text)


def clean_lyrics(letra):
    cleaned = ''
    for text in letra.split('\n')[1:]:   # remove the first line
        if text == '':
            continue
        text = re.sub(r'\[.+?\]', ' ', text) # remove whatever is between brackets
        text = re.sub(r'\((.+?)\)', r'\1', text) # keep what is between parentheses
        text = re.sub(r'\"(.+?)\"', r'\1', text) # keep what is between quotes
        text = re.sub(r'[\s]+', ' ', text) # spaces
        text = text.strip() # remove leading and trailing spaces
        text = text.lower() # lowercase
        cleaned += text + ' '
    cleaned = re.sub(r'\s+', ' ', cleaned) # remove multiple spaces
    cleaned = cleaned.strip() # remove leading and trailing spaces
    return ' '.join(remove_stuff(tok) for tok in cleaned.split(' '))

# Example Spanish documents
df = pd.read_csv('c:/Users/rmessina/Eli/data/song_lyrics_es_1950_2020.csv', encoding='utf8')  # Load your dataset

display(f"Loaded {len(df)} rows from the dataset")
display("Cleaning lyrics...")
# Apply multiprocessing to the DataFrame with a progress bar
df['cleaned_lyrics'] = df['lyrics'].swifter.apply(clean_lyrics)

# Remove NaN values from the 'lemmatized_lyrics' column
initial_count = len(df)
df = df.dropna(subset=['cleaned_lyrics'])
final_count = len(df)
removed_count = initial_count - final_count
display(f"Removed {removed_count} rows with NaN values from 'cleaned_lyrics'.")

df.to_csv('c:/Users/rmessina/Eli/data/song_lyrics_es_1950_2020_cleaned2.csv', encoding='utf8', index=False)

# Convert the cleaned column to a list
documents = df['cleaned_lyrics'].tolist()
process_col = 'cleaned_lyrics'



'Loaded 275059 rows from the dataset'

'Cleaning lyrics...'

Pandas Apply:   0%|          | 0/275059 [00:00<?, ?it/s]

"Removed 0 rows with NaN values from 'cleaned_lyrics'."

# For BERT, we should NOT need this... but results are horrible without!?

In [10]:
stopwords_set = set(stop_words)  # for faster lookup

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords_set])

# Apply to the column
df['no_stopwords'] = df['cleaned_lyrics'].swifter.apply(remove_stopwords)

initial_count = len(df)
df = df.dropna(subset=['no_stopwords'])
final_count = len(df)
removed_count = initial_count - final_count
display(f"Removed {removed_count} rows with NaN values from 'no_stopwords'.")

df.to_csv('c:/Users/rmessina/Eli/data/song_lyrics_es_1950_2020_cleaned_stop.csv', encoding='utf8', index=False)



Pandas Apply:   0%|          | 0/275059 [00:00<?, ?it/s]

"Removed 0 rows with NaN values from 'no_stopwords'."

In [12]:
import os
import time

if __name__ == "__main__":
    # Load a Spanish-compatible sentence embedding model
    embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device='cuda')

    # Convert the cleaned column to a list
    process_col = 'no_stopwords'
    documents = df[process_col].tolist()

    # Convert the cleaned column to a list
    np_emb = 'c:/Users/rmessina/Eli/data/lyrics_es_embeddings_nostop.npy'
    documents = df[process_col].tolist()
    if not os.path.isfile(np_emb):
        display(f"Saving embeddings to {np_emb}")
        embeddings = embedding_model.encode(documents, show_progress_bar=True)
        with open(np_emb, "wb") as f:
            np.save(f, embeddings)
    else:
        display(f"Loading embeddings from {np_emb}")
        with open(np_emb, "rb") as f:
            embeddings = np.load(f)

    try:
        for size in  [20, 30, 40, 50, 100]:  # [5, 10, 15, 25, 50, 100]:
            display(f"Doing {size}" + "^" * 37)
            hdbscan_model = HDBSCAN(min_cluster_size=size, min_samples=5, prediction_data=True)
            start = time.time()
            topic_model = BERTopic(
                embedding_model=embedding_model,
                hdbscan_model=hdbscan_model,          # Still used for clustering
                language="spanish",
                verbose=True,
                calculate_probabilities=False
            )

            topics, _ = topic_model.fit_transform(documents, embeddings)

            display(f"min_cluster_size={size} → {len(set(topics)) - (1 if -1 in topics else 0)} topics")
            topic_model.save(f"c:/Users/rmessina/Eli/models/bertopic_spanish_lyrics_lemma_umap_hdbscan{size}", save_embedding_model=True)

            display(f"Took {(time.time()-start)/60:.3f} minutes to process")

            df['topic'] = topics
            df.to_csv(f'c:/Users/rmessina/Eli/data/song_lyrics_es_1950_2020_bertopic_lemma_umap_hdbscan{size}.csv', encoding='utf8', index=False)

            # Print the topics
            display(topic_model.get_topic_info())
            # Display the top words for each topic

            display("*" * 42)
    except Exception as exc:
        pass


'Saving embeddings to c:/Users/rmessina/Eli/data/lyrics_es_embeddings_nostop.npy'

Batches:   0%|          | 0/8596 [00:00<?, ?it/s]

'Doing 20^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^'

2025-05-06 19:56:14,458 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-06 19:57:44,998 - BERTopic - Dimensionality - Completed ✓
2025-05-06 19:57:45,007 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-06 19:58:12,741 - BERTopic - Cluster - Completed ✓
2025-05-06 19:58:12,776 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-06 19:58:33,509 - BERTopic - Representation - Completed ✓


'min_cluster_size=20 → 815 topics'

...

'min_cluster_size=100 → 3 topics'

2025-05-06 20:12:34,514 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


'Took 3.030 minutes to process'

,Topic,Count,Name,Representation,Representative_Docs
0,-1,828,-1_que_el_la_me,"[que, el, la, me, en, te, amor, cuerpo, amo, no]",[ser aliado largas horas y el tiempo para pode...
1,0,272783,0_que_no_la_el,"[que, no, la, el, me, en, te, lo, mi, yo]",[con el tiempo todo pasa el dolor se tú yo que...
2,1,942,1_que_la_te_no,"[que, la, te, no, me, yo, el, lo, en, tú]",[this is the remix te llevo alto rafa pabön on...
3,2,506,2_konde_273_cris_estribillo,"[konde, 273, cris, estribillo, , , , , , ]","[, , 273]"


'******************************************'

In [25]:
from sklearn.cluster import KMeans


# Define the number of clusters (topics)
num_topics = 20  # You can tune this based on your data

# Initialize KMeans clustering model
kmeans = KMeans(n_clusters=num_topics, random_state=42)

# Initialize BERTopic with KMeans as the clustering model
topic_model = BERTopic(embedding_model=embedding_model, 
                       hdbscan_model=kmeans,
                       language="spanish")  # Set language if needed


# Fit the BERTopic model with your documents
topics, probabilities = topic_model.fit_transform(documents, embeddings)

# Save the model
topic_model.save(f"c:/Users/rmessina/Eli/models/bertopic_kmeans_model_{num_topics}_topics")

# Add topics to your dataframe
df['topic'] = topics
df.to_csv(f'c:/Users/rmessina/Eli/data/song_lyrics_es_1950_2020_bertopic_kmeans_{num_topics}_topics.csv', encoding='utf8', index=False)


2025-05-06 21:28:53,728 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [26]:
import plotly.express as px
from umap import UMAP


# Step 2: Reduce with UMAP
umap_model = UMAP(n_components=2, random_state=42)
reduced = umap_model.fit_transform(embeddings)

# Step 3: Create dataframe
viz_df = pd.DataFrame(reduced, columns=['x', 'y'])
viz_df['topic'] = topics
viz_df['decade'] = df['decade']

# Step 4: Plot with Plotly
fig = px.scatter(
    viz_df, x='x', y='y',
    color=viz_df['topic'].astype(str),
    symbol='decade',
    title="Topics visualized via UMAP",
    labels={'color': 'Topic', 'symbol': 'Decade'},
    opacity=0.7
)
fig.show()

# Swap the embedding model: redo this without the "stopwords" removed!

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("hiiamsid/sentence_similarity_spanish_es")

topic_model = BERTopic(embedding_model=embedding_model, language="spanish")

umap_model = UMAP(n_components=5, random_state=42)
kmeans_model = KMeans(n_clusters=50, random_state=42)

topic_model = BERTopic(
    embedding_model=embedding_model,
    language="spanish",
    umap_model=umap_model,
    hdbscan_model=kmeans_model  # note: param is still called `hdbscan_model`
)

docs = df['cleaned_lyrics'].tolist()
topics, probs = topic_model.fit_transform(docs)

topic_model.visualize_topics()



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
topic_model.save(f"c:/Users/rmessina/Eli/models/bertopic_kmeans_model_hiiamsid_{num_topics}_topics")

2025-05-06 22:37:01,783 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
